In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [7]:
data=pd.read_csv("iris.csv")
# data.head()
# print("Columns Rows")
# print("____________")
# data.shape
#drop duplicates except fist
data = data.drop_duplicates(keep='first')
#checking duplicate values
data.duplicated().sum()
data['species'].unique()
encoder = LabelEncoder()
data['species'] = encoder.fit_transform(data.species)
x = data.drop('species',axis=1)
y = data['species']
data
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
from sklearn.feature_selection import SelectKBest, chi2

# Apply feature selection using SelectKBest with chi-squared test
k = 2  # Number of features to select
selector = SelectKBest(chi2, k=k)
X_new = selector.fit_transform(x, y)

# Get the selected feature indices
selected_feature_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = [data.species[i] for i in selected_feature_indices]

# Print the selected features
print("Selected Features:")
for feature in selected_feature_names:
    print(feature)
    
    
model = LogisticRegression()
model.fit(x_train,y_train)
train_acc = model.score(x_train,y_train)
test_acc = model.score(x_test,y_test)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_pred, y_test)

print(f'Train Accuracy:{train_acc}')
print(f'Test Accuracy:{test_acc}')
print(f'Accuracy:{accuracy}')

Selected Features:
0
0
Train Accuracy:0.9803921568627451
Test Accuracy:0.9777777777777777
Accuracy:0.9777777777777777


In [8]:
new_data = [ [2.1, 3.0, 4.7, 1.0],[5.1, 3.5, 1.4, 0.2],  # example features of a new iris flower
            [6.2, 2.9, 4.3, 1.3]]

new_data_scaled = scaler.transform(new_data)
predictions = model.predict(new_data_scaled)

# Decode the predictions back to original species names
predicted_species = encoder.inverse_transform(predictions)

print("Predicted Species:")
for species in predicted_species:
    print(species)


Predicted Species:
setosa
setosa
versicolor


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'logistic_regression_model.pkl')


['logistic_regression_model.pkl']

In [10]:
import joblib
import numpy as np

# Load the saved model
model = joblib.load('logistic_regression_model.pkl')

# Preprocess the new data
new_data = np.array([[2.1, 5.6,6.2, 3.2], [5.1, 3.5, 1.4, 0.2], [6.2, 2.9, 4.3, 1.3]])
new_data_scaled = scaler.transform(new_data)  # Assuming 'scaler' is the StandardScaler used for preprocessing

# Make predictions on the new data
predictions = model.predict(new_data_scaled)

# Print the predictions
print(predictions)


[0 0 1]


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [11]:
# predicted_class = encoder.inverse_transform([2])[0]
# print(predicted_class)
# Assuming 'encoder' is the LabelEncoder used for encoding the target variable
predicted_class = encoder.inverse_transform(predictions)
print(predicted_class)


['setosa' 'setosa' 'versicolor']


In [12]:
import os
print(os.path.getsize('logistic_regression_model.pkl'))


855


### FASTAPI

In [13]:
import uvicorn
from BankNotes import BankNote
from fastapi import FastAPI
import numpy as np
import pickle
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

In [14]:
# Load the trained model
model = joblib.load('logistic_regression_model.pkl')
# scaler = StandardScaler()  # Assuming 'scaler' is the StandardScaler used for preprocessing

# Create a FastAPI app
app = FastAPI()

# Define a Pydantic model for the request body
class IrisData(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

# Define a route for the prediction endpoint
@app.post("/predict")
def predict_species(data: IrisData):
    try:
        # Prepare the data for prediction
        new_data = np.array([[data.sepal_length, data.sepal_width, data.petal_length, data.petal_width]])
        new_data_scaled = scaler.transform(new_data)

        # Make the prediction
        prediction = model.predict(new_data_scaled)

        # Return the predicted class
        return {"predicted_species": int(prediction[0])}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
import nest_asyncio
import uvicorn
nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)

INFO:     Started server process [11316]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


INFO:     ::1:56979 - "GET / HTTP/1.1" 404 Not Found
INFO:     ::1:56980 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:56980 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     ::1:56981 - "POST /predict HTTP/1.1" 200 OK


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


INFO:     ::1:57000 - "GET / HTTP/1.1" 404 Not Found
INFO:     ::1:57001 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:57001 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     ::1:57003 - "POST /predict HTTP/1.1" 200 OK


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
